## Notebook for the Viszualization and Calculation of BIC (2H Data)

ToDo:
    - Documentation
    - fix paths

In [1]:
import pandas as pd
import os
import errno  # handy system and path functions
import sys  # to get file system encoding
import glob
import locale
import numpy as np
import matplotlib.pyplot as plt
import scipy.linalg
import seaborn as sns

from mpl_toolkits.mplot3d import Axes3D
from matplotlib.lines import Line2D
import matplotlib.patches as mpatches
from matplotlib import rcParams
params = {'legend.fontsize': 20,
          'legend.handlelength': 2,
         'figure.autolayout': True}
rcParams.update(params)
from scipy import signal
from scipy import stats
from sklearn import preprocessing

# np.set_printoptions(precision=28)
# pd.set_option('precision', 28)


# sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})
sns.set_context("paper")
sns.set_style('whitegrid')
sns.set_palette('colorblind')

/home/michael/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
# # Set the style globally

# plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = 'Ubuntu'
# plt.rcParams['font.monospace'] = 'Ubuntu Mono'
# plt.rcParams['font.size'] = 10
# plt.rcParams['axes.labelsize'] = 10
# plt.rcParams['axes.labelweight'] = 'bold'
plt.rcParams['axes.titlesize'] = 16
plt.rcParams['xtick.labelsize'] = 14
plt.rcParams['ytick.labelsize'] = 14
plt.rcParams['legend.fontsize'] = 14
# plt.rcParams['figure.titlesize'] = 12
plt.rcParams['figure.dpi'] = 300

In [3]:
%run ba_analysis_2h_functions.py

In [4]:
marker_dict

{0: 'head_1',
 1: 'head_2',
 2: 'head_3',
 3: 'head_4',
 4: 'left_hand_1',
 5: 'left_hand_2',
 6: 'left_hand_3',
 7: 'right_hand_1',
 8: 'right_hand_2',
 9: 'right_hand_3',
 10: 'chest',
 11: 'stomach',
 12: 'upper_back_central',
 13: 'lower_back_central',
 14: 'right_back_hip',
 15: 'left_back_hip',
 16: 'left_shoulder',
 17: 'left_upper_arm',
 18: 'left_lower_arm',
 19: 'left_wrist',
 20: 'right_shoulder',
 21: 'right_upper_arm',
 22: 'right_lower_arm',
 23: 'right_wrist',
 24: 'left_foot_1',
 25: 'left_foot_2',
 26: 'left_foot_3',
 27: 'left_lower_leg',
 28: 'left_knee',
 29: 'left_upper_leg',
 30: 'left_front_hip',
 31: 'right_front_hip',
 32: 'right_upper_leg',
 33: 'right_knee',
 34: 'right_lower_leg',
 35: 'right_foot_1',
 36: 'right_foot_2',
 37: 'right_foot_3'}

In [5]:
def get_model(dimensions, event_file, data_file, hand):
    ''' 
    - Get events from event csv and and extract movement corresponding to events from data_file
    - extract mean & variance for each physical dimension
    - correct movement data by means as to extract only positional variability over time
    - reshape movement data into form for svd
    '''
    df_model_a = pd.DataFrame()
    print(dimensions)

    for i in dimensions:
        print('!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
        print(marker_dict[i])
        i = str(i)

        # marker = ['marker_7_x', 'marker_7_y', 'marker_7_z']
        marker = ['marker_'+i+'_x', 'marker_'+i+'_y', 'marker_'+i+'_z']
#         hand = 'r'
        # extract and resample movements
        df_x, df_y, df_z = extract_and_resample_2H(event_file, data_file, marker, hand)
        # extract means, variance and mean-corrected dataframe of movements
        x_mean_dimension, x_variance_dimension, x_df_corrected = get_corrected('x', df_x)
        y_mean_dimension, y_variance_dimension, y_df_corrected = get_corrected('y', df_y)
        z_mean_dimension, z_variance_dimension, z_df_corrected = get_corrected('z', df_z)

        # concatenate mean-corrected movements into samples*trials rows and markers(for each dim) as columns
        reshaped_df = reshape(x_df_corrected, y_df_corrected, z_df_corrected)
        for idx, j in enumerate(reshaped_df.columns):
            df_model_a[str(marker[idx])] = reshaped_df[j]
    return df_model_a

In [6]:
def get_model_2H(dimensions, event_file, data_file):
    ''' 
    - Get events from event csv and and extract movement corresponding to events from data_file
    - extract mean & variance for each physical dimension
    - correct movement data by means as to extract only positional variability over time
    - reshape movement data into form for svd
    '''
    df_model_a = pd.DataFrame()
    print(dimensions)

    for i in dimensions:
        print('!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
        print(marker_dict[i])
        if (marker_dict[i].split('_')[0]) == 'right':
            print('here')
            hand = 'r'
            i = str(i)

            # marker = ['marker_7_x', 'marker_7_y', 'marker_7_z']
            marker = ['marker_'+i+'_x', 'marker_'+i+'_y', 'marker_'+i+'_z']
    #         hand = 'r'
            # extract and resample movements
            df_x, df_y, df_z = extract_and_resample_2H(event_file, data_file, marker, hand)
            # extract means, variance and mean-corrected dataframe of movements
            x_mean_dimension, x_variance_dimension, x_df_corrected = get_corrected('x', df_x)
            y_mean_dimension, y_variance_dimension, y_df_corrected = get_corrected('y', df_y)
            z_mean_dimension, z_variance_dimension, z_df_corrected = get_corrected('z', df_z)

            # concatenate mean-corrected movements into samples*trials rows and markers(for each dim) as columns
            reshaped_df = reshape(x_df_corrected, y_df_corrected, z_df_corrected)
            for idx, j in enumerate(reshaped_df.columns):
                df_model_a[str(marker[idx])] = reshaped_df[j]
        elif (marker_dict[i].split('_')[0]) == 'left':
            print('here')
            hand = 'l'
            i = str(i)

            # marker = ['marker_7_x', 'marker_7_y', 'marker_7_z']
            marker = ['marker_'+i+'_x', 'marker_'+i+'_y', 'marker_'+i+'_z']
    #         hand = 'r'
            # extract and resample movements
            df_x, df_y, df_z = extract_and_resample_2H(event_file, data_file, marker, hand)
            
            # extract means, variance and mean-corrected dataframe of movements
            x_mean_dimension, x_variance_dimension, x_df_corrected = get_corrected('x', df_x)
            y_mean_dimension, y_variance_dimension, y_df_corrected = get_corrected('y', df_y)
            z_mean_dimension, z_variance_dimension, z_df_corrected = get_corrected('z', df_z)

            # concatenate mean-corrected movements into samples*trials rows and markers(for each dim) as columns
            reshaped_df = reshape(x_df_corrected, y_df_corrected, z_df_corrected)
            for idx, j in enumerate(reshaped_df.columns):
                df_model_a[str(marker[idx])] = reshaped_df[j]
    return df_model_a

In [7]:
def modelComparison(data):
    """Decompose data with SVD and compute BIC-like approximation to log(P(data|number of primitives))
    for each possible number of primitives
    returns: variance-accounted-for and log P for each number of synergies"""
    
    U,S,VT=np.linalg.svd(data)
    
    SVT=np.diag(S).dot(VT)
    
    tot_var=data.var() # this may have to be computed per trial, i.e. as data.var(axis=1).mean()
    N=data.size
    
    VAF=[]
    logPD=[]
    
    for num_prim in range(len(SVT)-2): # -2 to avoid 
        
        sub_u=U[:,:num_prim+1]
        sub_svt=SVT[:num_prim+1]
        reconstruction=np.dot(sub_u,sub_svt)
        
        tot_num_df=sub_u.size+sub_svt.size # total number of degrees of freedom in the model
        
        rec_err_2= ((data-reconstruction)**2).mean()
        VAF.append(1.0-rec_err_2/tot_var)
        log_likelihood=-N/2*(np.log(2*np.pi)+np.log(rec_err_2)+1)
        logPD.append(log_likelihood-0.5*tot_num_df*np.log(N))
        
    return VAF,np.array(logPD)

In [8]:
def plot_loadings(VT, model):
    plt.rcParams["figure.figsize"] = (10,5)
    print(VT.shape)

    counter = 0
    labels = []

    plt.plot(range(len(VT[0,:])),VT[0,:] )
    plt.ylim(-1,1)
    plt.xticks(np.arange(len(VT[0,:])), labels, rotation=70)
    plt.title('Singular vector v1 loadings')
    plt.show()
    
    sns.distplot(VT[0,:], rug=True, kde=True);
    plt.ylim(0,3)
    plt.title('Density of vt1 loadings')
    plt.ylabel('density')
    plt.show()

    return VT



# parameters = [0,0,0,1,1,1,1,1,1,1,1,1,1,1,1]
def specify_model(model, parameters):
    model_parameters = []
    for idx, i in enumerate(parameters):  
        if i == 1:
#             print(model.columns[idx])
            model_parameters.append(model.columns[idx])
    return model_parameters
    

def extract_mean_factor_loading(reshaped, path_to_storage):
    '''
    Perform SVD on Data generated by "reshaped"-function extract mean loading
    and standard deviation of mean of first singular vector of U-matrix.
    Concatenate mean_loadings for each trial into single column matrix.

    input:
    reshaped: Dataframe with x,y, z dimensions as columns and
    sample*trials as rows

    output:
    U = n*n matrux containing left singular vectors
    S = m*m diagonal matrix containing singular values
    VT = m*m matrix containing right singular vectors
    mean_loadings = mean loadings on u1 over all trials
    stdm = standard deviation of mean loadings
    concatenated = dataframe containing mean loadings; trial*samples long

    '''
    U, s, VT = scipy.linalg.svd(reshaped, full_matrices=False)
    S = np.diag(s)

    fig = plt.figure(figsize=(14,7))
    len_x = []
    for i in range(1,len(s)+1):
        print(i)
        len_x.append(i)

    sns.barplot(y=s, x=len_x)
    #plt.ylim(0,4)
    plt.title('Singular Values:')
    fig.savefig(path_to_storage+'_singular_values.png')
    plt.show()

    
    # show explained variance
    fig = plt.figure(figsize=(14,7))    
    var_explained = np.round(s**2/np.sum(s**2), decimals=3)
    #print(type(var_explained))
    legend_elements = []
    for idx, i in enumerate(var_explained):
        element = Line2D([0], [0], alpha=0, lw=2, marker='o', markersize=15, label=('SV '+str(idx)+': '+str(var_explained[idx])))
        legend_elements.append(element)
    
    
    sns.barplot(x=list(range(1,len(var_explained)+1)),
            y=var_explained)
    plt.xlabel('Singular Values', fontsize=16)
    plt.ylabel('Variance Explained (%)', fontsize=16)
    plt.ylim(0,1)
    plt.legend(handles=legend_elements, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., prop={'size': 10})
    fig.savefig(path_to_storage+'_singular_var_explained.png')
    plt.show()

    # reshape to easier calculate mean loadings
    counter = 0
    trial_length = 300
    concatenated = pd.DataFrame(columns=range(len(reshaped)//trial_length))
    for i in range(len(reshaped)//trial_length):

        counter +=trial_length
        if i == 0:
#             print(U[:counter,0])  # take only first column of U matrix -> first principal component
            concatenated[concatenated.columns[i]] = U[:counter,0]
        elif i >0:
            concatenated[concatenated.columns[i]] = U[counter-trial_length:counter,0]

    # transform trial to matrix and transform each point to absolute value
    concatenated = concatenated.values
    concatenated = np.absolute(concatenated)

    # extract mean loading and standarad error of mean
    mean_loadings = []
    stdm = []
    for i in range(trial_length):
        mean_loadings.append(concatenated[i].mean())
        stdm.append(scipy.stats.sem(concatenated[i]))

    mean_loadings = np.array(mean_loadings)
    stdm = np.array(stdm)
    #print(s)
    return U, s, S, VT, mean_loadings, stdm, concatenated, var_explained

In [10]:
dir_ = '/home/michael/bachelorarbeit/subjects/*'
df_r1 = pd.DataFrame()
path = '/home/michael/Desktop/model_evd/2H/'
subjects_info = pd.DataFrame()
for subject in (glob.glob(dir_)):
    subject_id = subject.split('/')[5]
    print(subject_id)
    try:
        os.makedirs(path+subject_id+'/')
    except OSError as e:
        if e.errno != errno.EEXIST:
            raise

    if subject_id == 'sub-02':
        print('bad-subject')
#         break
    elif subject_id == 'sub-03':
        print('bad-subject')
#         break#

#     else:
    for filename in (glob.glob(subject+'/*')):
#         print(filename)
        if filename.split('_')[2] == 'parameters':
            print('info')
            subject_parameters = pd.read_csv(filename)
            subjects_info = subjects_info.append(subject_parameters)
        elif filename.split('-')[3] == '2H' and filename.split('-')[4] == 'events':
            print('events')
            print(filename)
            event_file_2H = pd.read_csv(filename)
        elif filename.split('-')[3] == '2H' and filename.split('-')[4] == 'data':
            print('data')
            print(filename)
            data_file_2H = pd.read_csv(filename)

    ## RIGHT HAND
    r1,r2,r3,m1_r,m2_r,m3_r,_,_,_ = group_by_position_2H(event_file_2H, 'r')

#     _,_,_,m1_l,m2_l,m3_l,l1,l2,l3 = group_by_position_2H(event_file_2H, 'l')
    positions_list = [r1,r2,r3,m1_r,m2_r,m3_r]
    pos_names = ['r1','r2','r3','m1_r','m2_r','m3_r']
    for idx, i in enumerate(positions_list):
        print(pos_names[idx])
        try:
            os.makedirs(path+subject_id+'/'+pos_names[idx]+'/')
        except OSError as e:
            if e.errno != errno.EEXIST:
                raise
        # define marker space
        dimensions = right_hand_marker[:1] + right_wrist + right_lower_arm  + right_upper_arm + right_shoulder 

        model = get_model(dimensions, i, data_file_2H, hand='r')
        # specify which parameters to extract
        parameters = [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]

        model_parameters = specify_model(model, parameters)
        model = model[model_parameters]

        # SVD
        path_to_storage = path+subject_id+'/'+pos_names[idx]+'/'+subject_id+'_'+pos_names[idx]


        ## PLOT MODEL EVIDENCE RIGHT HAND
        VAF,logPD = modelComparison(np.array(model))

        df_vaf = pd.DataFrame(VAF)
        df_vaf.to_csv(path+subject_id+'/'+pos_names[idx]+'/'+subject_id+'_'+pos_names[idx]+'_'+'_right_vaf.csv', index=False, encoding='utf-8')

        model_evd = pd.DataFrame(logPD-logPD.min())
        model_evd.to_csv(path+subject_id+'/'+pos_names[idx]+'/'+subject_id+'_'+pos_names[idx]+'_model_evidence.csv', index=False, encoding='UTF-8')


    ## LEFT HAND
    _,_,_,m1_l,m2_l,m3_l,l1,l2,l3 = group_by_position_2H(event_file_2H, 'l')
    positions_list_r = [m1_l,m2_l,m3_l,l1,l2,l3 ]
    pos_names = ['m1_l','m2_l','m3_l','l1','l2','l3' ]
    for idx, i in enumerate(positions_list):
        print(pos_names[idx])

        try:
            os.makedirs(path+subject_id+'/'+pos_names[idx]+'/')
        except OSError as e:
            if e.errno != errno.EEXIST:
                raise
        dimensions = left_hand_marker[:1] + left_wrist + left_lower_arm  + left_upper_arm + left_shoulder 

        model = get_model(dimensions, i , data_file_2H, 'l')
        # specify which parameters to extract
        parameters = [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]

        model_parameters = specify_model(model, parameters)
        model = model[model_parameters]
        # SVD
        path_to_storage = path+subject_id+'/'+pos_names[idx]+'/'+subject_id+'_'+pos_names[idx]

        VAF,logPD = modelComparison(np.array(model))

        df_vaf = pd.DataFrame(VAF)
        df_vaf.to_csv(path+subject_id+'/'+pos_names[idx]+'/'+subject_id+'_'+pos_names[idx]+'_'+'_left_vaf.csv', index=False, encoding='utf-8')

        model_evd = pd.DataFrame(logPD-logPD.min())
        model_evd.to_csv(path+subject_id+'/'+pos_names[idx]+'/'+subject_id+'_'+pos_names[idx]+'_model_evidence.csv', index=False, encoding='UTF-8')

        plt.show()

    r1_m1,r1_l1,m1_l1,r2_l2,m2_l2,r2_m2,r3_l3,m3_l3,r3_m3,r1_l3,r3_l1,m1_m3,m3_m1 = group_by_pos_both_hands(event_file_2H)
    positions_list = [r1_m1,r1_l1,m1_l1,r2_l2,m2_l2,r2_m2,r3_l3,m3_l3,r3_m3,r1_l3,r3_l1,m1_m3,m3_m1]
    pos_names = ['r1_m1','r1_l1','m1_l1','r2_l2','m2_l2','r2_m2','r3_l3','m3_l3','r3_m3','r1_l3','r3_l1','m1_m3','m3_m1']

    ## BOTH HANDS
    for idx, i in enumerate(positions_list):
        print(pos_names[idx])


        try:
            os.makedirs(path+subject_id+'/'+pos_names[idx]+'/')
        except OSError as e:
            if e.errno != errno.EEXIST:
                raise

        dimensions = right_hand_marker[:1] + right_wrist + right_lower_arm  + right_upper_arm + right_shoulder + left_hand_marker[:1] + left_wrist + left_lower_arm  + left_upper_arm + left_shoulder

        model = get_model_2H(dimensions, i, data_file_2H)
        # specify which parameters to extract
        parameters = [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]


        model_parameters = specify_model(model, parameters)
        model = model[model_parameters]

        # SVD
        path_to_storage = path+subject_id+'/'+pos_names[idx]+'/'+subject_id+'_'+pos_names[idx]

        VAF,logPD = modelComparison(np.array(model))

        df_vaf = pd.DataFrame(VAF)
        df_vaf.to_csv(path+subject_id+'/'+pos_names[idx]+'/'+subject_id+'_'+pos_names[idx]+'_'+'_left_&_right_vaf.csv', index=False, encoding='utf-8')

        model_evd = pd.DataFrame(logPD-logPD.min())
        model_evd.to_csv(path+subject_id+'/'+pos_names[idx]+'/'+subject_id+'_'+pos_names[idx]+'_model_evidence.csv', index=False, encoding='UTF-8')

sub-07
sub-07
data
/home/michael/bachelorarbeit/subjects/sub-07/sub-07-2H-data-2019_09_11_12_59.csv
info
events
/home/michael/bachelorarbeit/subjects/sub-07/sub-07-2H-events-2019_09_11_12_59.csv
r1
[7, 23, 22, 21, 20]
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
right_hand_1


/home/michael/.local/lib/python3.6/site-packages/scipy/signal/signaltools.py:2223: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  Y[sl] = X[sl]
/home/michael/.local/lib/python3.6/site-packages/scipy/signal/signaltools.py:2225: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  Y[sl] = X[sl]
/home/michael/.local/lib/python3.6/site-packages/scipy/signal/signaltools.py:2233: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as a

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
right_wrist
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
right_lower_arm
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
right_upper_arm
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
right_shoulder
r2
[7, 23, 22, 21, 20]
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
right_hand_1
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
right_wrist
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
right_lower_arm
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
right_upper_arm
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
right_shoulder
r3
[7, 23, 22, 21, 20]
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
right_hand_1
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
right_wrist
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
right_upper_arm
here
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
right_shoulder
here
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
left_hand_1
here
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
left_wrist
here
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
left_lower_arm
here
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
left_upper_arm
here
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
left_shoulder
here
m2_l2
[7, 23, 22, 21, 20, 4, 19, 18, 17, 16]
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
right_hand_1
here
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
right_wrist
here
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
right_lower_arm
here
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
right_upper_arm
he

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
right_upper_arm
here
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
right_shoulder
here
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
left_hand_1
here
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
left_wrist
here
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
left_lower_arm
here
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
left_upper_arm
here
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
left_shoulder
here
sub-01
sub-01
data
/home/michael/bachelorarbeit/subjects/sub-01/sub-01-2H-data-2019_09_09_11_29.csv
info
events
/home/michael/bachelorarbeit/subjects/sub-01/sub-01-2H-events-2019_09_09_11_29.csv
r1
[7, 23, 22, 21, 20]
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
right_hand_1
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
right_wrist
!!!!!!!!!!!!

...

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
left_hand_1
here
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
left_wrist
here
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
left_lower_arm
here
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
left_upper_arm
here
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
left_shoulder
here
sub-10
sub-10
info
data
/home/michael/bachelorarbeit/subjects/sub-10/sub-10-2H-data-2019_09_13_13_04.csv
events
/home/michael/bachelorarbeit/subjects/sub-10/sub-10-2H-events-2019_09_13_13_04.csv
r1
[7, 23, 22, 21, 20]
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
right_hand_1
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
right_wrist
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
right_lower_arm
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
right_upper_arm
!!!!!!!!!!!!!!!!!!!!!

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
left_hand_1
here
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
left_wrist
here
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
left_lower_arm
here
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
left_upper_arm
here
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
left_shoulder
here
r2_l2
[7, 23, 22, 21, 20, 4, 19, 18, 17, 16]
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
right_hand_1
here
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
right_wrist
here
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
right_lower_arm
here
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
right_upper_arm
here
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
right_shoulder
here
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
left_hand_1
here
!

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
left_hand_1
here
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
left_wrist
here
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
left_lower_arm
here
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
left_upper_arm
here
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
left_shoulder
here
m3_m1
[7, 23, 22, 21, 20, 4, 19, 18, 17, 16]
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
right_hand_1
here
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
right_wrist
here
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
right_lower_arm
here
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
right_upper_arm
here
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
right_shoulder
here
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
left_hand_1
here
!

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
left_hand_1
here
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
left_wrist
here
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
left_lower_arm
here
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
left_upper_arm
here
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
left_shoulder
here
m1_l1
[7, 23, 22, 21, 20, 4, 19, 18, 17, 16]
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
right_hand_1
here
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
right_wrist
here
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
right_lower_arm
here
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
right_upper_arm
here
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
right_shoulder
here
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
left_hand_1
here
!

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
left_hand_1
here
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
left_wrist
here
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
left_lower_arm
here
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
left_upper_arm
here
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
left_shoulder
here
m1_m3
[7, 23, 22, 21, 20, 4, 19, 18, 17, 16]
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
right_hand_1
here
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
right_wrist
here
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
right_lower_arm
here
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
right_upper_arm
here
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
right_shoulder
here
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
left_hand_1
here
!